# Tutorial and demo of core MEEGsim features

In [ ]:
import mne

from mne.datasets import sample

from meegsim.location import select_random
from meegsim.waveform import narrowband_oscillation
from meegsim.simulate import SourceSimulator

from meegsim_tutorial.utils import FILL_ME
from meegsim_tutorial.viz import show_sources

## 0. Configuration

Below, provide the same path to the sample dataset that you used during the installation check.

In [ ]:
download_path = None
# download_path = FILL_ME(
#     "Provide the same path (or use None) that used during the installation check."
# )

## 1. Navigating the source space

Source space is one of the key ingredients in every simulation, since it defines location and orientations of all sources that we model. In this section, we have a closer look at how one can navigate the source space. 

**Take home:** in MEEGsim, we use two numbers to define position of sources that are added to the simulation: 
 * index of the source space (0 - left hemisphere, 1 - right hemisphere) 
 * index of the vertex within the respective hemisphere (`vertno`).

Let's create a source space using the template MPI (`fsaverage`) from the sample MNE dataset and inspect it first. The `spacing` parameter below defines how coarse the source space. We recommend using `oct5` throughout the tutorial to reduce the computational load during the workshop. Find more about other possible and recommended spacing values [here](https://mne.tools/stable/documentation/cookbook.html#setting-up-the-source-space).

In [ ]:
data_path = sample.data_path(path=download_path)
subject = "fsaverage"
subjects_dir = data_path / "subjects"
src = mne.setup_source_space(
    subject=subject,
    spacing="oct5",
    subjects_dir=subjects_dir,
    add_dist=False
)

If we print the resulting `src`, we get an overview of the generated source space: two hemispheres with 1026 vertices in each hemisphere. Hemisphere-specific data can be accessed with `src[0]` and  `src[1]` for left and right hemisphere, respectively.

In [ ]:
print(src)

In [ ]:
src[0]

It is also possible to visualize the positions of all sources on the brain surface. Each source is a dipole that corresponds to a group of aligned pyramidal neurons:

In [ ]:
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    surfaces="white",
    coord_frame="mri",
    src=src,
)

Each vertex has a unique number between 0 and 163841. The `vertno` array stored indices of all vertices that belong to the source space (in this case, the left hemisphere):

In [ ]:
src[0]["vertno"]

In MEEGsim, we use the combination of the hemisphere index (0/1) and `vertno` to define the position of added sources. You can try it out below in a small example (not yet simulation). The helper function `show_source` will show the source that is defined by `hemi_idx` and `vertno` below.

In [ ]:
hemi_idx = 0
vertno = 163694

In [ ]:
# TIP: add surf="pial" or surf="pial_semi_inflated" to show sulci/gyri
show_sources([(hemi_idx, vertno)], subjects_dir)

**EXERCISES**:
1. Try moving the source to the right hemisphere.
2. Try changing the `vertno` value to select a source in frontal/occipital/your favorite area.

Sometimes, it can be useful to choose vertices randomly, and the [`select_random`](https://meegsim.readthedocs.io/en/latest/generated/meegsim.location.select_random.html) function can be used for this purpose. Notice the format of its output (pairs of numbers):

In [ ]:
select_random(src, n=10)

In [ ]:
show_sources(select_random(src, n=10), subjects_dir)

## 2. Forward model

## 3. Generating source activity

In [ ]:
sim = SourceSimulator(src)
sim.add_noise_sources(location=select_random, location_params=dict(n=100))
sim.add_point_sources(
    location=select_random,
    location_params=dict(n=3),
    waveform=narrowband_oscillation,
    waveform_params=dict(fmin=8, fmax=12),
)
sim.add_patch_sources(
    location=select_random,
    location_params=dict(n=3),
    waveform=narrowband_oscillation,
    waveform_params=dict(fmin=8, fmax=12),
    extents=15,
    subject="fsaverage",
    subjects_dir=subjects_dir,
)

In [ ]:
sc = sim.simulate(sfreq=250, duration=120)

In [ ]:
sc._sources

In [ ]:
sc._noise_sources

In [ ]:
sc.plot(subject="fsaverage", subjects_dir=subjects_dir)